In [2]:
API_KEY = "PER4V6RYCAU4TZ54M69D9GXHWNVKX6DY7Z"
BASE_URL = "https://api.etherscan.io/api"
ETHER_VALUE = 10 ** 18
address = "0x73bceb1cd57c711feac4224d062b0f6ff338501e"
address2 = "0xddbd2b932c763ba5b1b7ae3b362eac3e8d40121a"

In [3]:
len("0x73bceb1cd57c711feac4224d062b0f6ff338501e")

42

In [4]:
import pandas as pd
from requests import get
from matplotlib import pyplot as plt
from datetime import datetime

import matplotlib.ticker as mticker
from matplotlib.gridspec import GridSpec # 畫圖網格佈局

In [174]:
def make_sourcecode_api_url(address, **kwargs):
    url = BASE_URL + f"?module=contract&action=getsourcecode&address={address}&apikey={API_KEY}"

    for key, value in kwargs.items():
        url += f"&{key}={value}"

    return url

In [235]:
contract_address = "0x283af0b28c62c092c9727f1ee09c02ca627eb7f5"

In [236]:
responce = get(make_sourcecode_api_url(contract_address)).json()

In [241]:
def get_contract_name(contract_address):
    responce = get(make_sourcecode_api_url(contract_address)).json()
    name = responce["result"][0]["ContractName"]
    return name

In [237]:
responce["result"][0].keys()

dict_keys(['SourceCode', 'ABI', 'ContractName', 'CompilerVersion', 'OptimizationUsed', 'Runs', 'ConstructorArguments', 'EVMVersion', 'Library', 'LicenseType', 'Proxy', 'Implementation', 'SwarmSource'])

In [239]:
name = responce["result"][0]["ContractName"]

In [240]:
name

'ETHRegistrarController'

### Build up class

In [169]:
class Get_address_details():
    
    def __init__(self, address):
       
        self.address = address
        self.BASE_URL = "https://api.etherscan.io/api"
        self.ETHER_VALUE = 10 ** 18
        self.API_KEY = "PER4V6RYCAU4TZ54M69D9GXHWNVKX6DY7Z"
        
    def make_api_url(self, module, action, **kwargs):
        url = self.BASE_URL + f"?module={module}&action={action}&address={self.address}&apikey={self.API_KEY}"

        for key, value in kwargs.items():
            url += f"&{key}={value}"

        return url
    
    def make_hash_api_url(self, module, action, txhash, **kwargs):
        url = self.BASE_URL + f"?module={module}&action={action}&txhash={txhash}&apikey={self.API_KEY}"
        
        for key, value in kwargs.items():
            url += f"&{key}={value}"

        return url
    
    def get_txns_from_txnhash(self, txhash):
        txn_url = self.make_hash_api_url("account", "txlistinternal", txhash)
        responce = get(txn_url)
        data = responce.json()
        
        return data["result"]
        
    def get_account_balance(self):
        balance_url = self.make_api_url("account", "balance", tag="latest")
        response = get(balance_url)
        data = response.json()

        value = int(data["result"]) / ETHER_VALUE
        return value
    
    def get_all_txns(self):
        """
        Get all txns 
        """
        transactions_url = self.make_api_url("account", "txlist", startblock=0, endblock=99999999, page=1, offset=10000, sort="asc")
        response = get(transactions_url)
        nor_data = response.json()["result"]
        nor_df = pd.DataFrame(nor_data)
        nor_df.insert(1, "txn_type", value ="normal")
        
        internal_tx_url = self.make_api_url("account", "txlistinternal", startblock=0, endblock=99999999, page=1, offset=10000, sort="asc")
        in_response = get(internal_tx_url)
        in_data = in_response.json()["result"]
        in_df = pd.DataFrame(in_data)
        in_df.insert(1, "txn_type", value = "internal")
        
        data_df = pd.concat([nor_df, in_df], join = "outer")
        
        new_time_list = []
        for idx, timestamp in enumerate(data_df["timeStamp"]):
            data_df["timeStamp"][idx] = int(timestamp)
            new_time = str(datetime.fromtimestamp(int(timestamp))).split(" ")[0]
            new_time_list.append(new_time)
        data_df.insert(1, "datetime", new_time_list)
        data_df["datetime"] = pd.to_datetime(data_df["datetime"], format="%Y-%m-%d")
        date_df_out = data_df.set_index("datetime")
        # data_df.reset_index(inplace=True, drop=True)
        
        return date_df_out
    
    def get_internal_txns(self):
        internal_tx_url = self.make_api_url("account", "txlistinternal", startblock=0, endblock=99999999, page=1, offset=10000, sort="asc")
        in_response = get(internal_tx_url)
        in_data = in_response.json()["result"]
        # data_df = pd.DataFrame(in_data)

        # new_time_list = []
        # for idx, timestamp in enumerate(data_df["timeStamp"]):
        #     data_df["timeStamp"][idx] = int(timestamp)
        #     new_time = str(datetime.fromtimestamp(int(timestamp))).split(" ")[0]
        #     new_time_list.append(new_time)
        # data_df.insert(1, "datetime", new_time_list)
        # data_df["datetime"] = pd.to_datetime(data_df["datetime"], format="%Y-%m-%d")
        # date_df_out = data_df.set_index("datetime")
        # data_df.reset_index(inplace=True, drop=True)
        
        return in_data

In [170]:
def get_txn_counts(address_df, txn_type, txn_time):

    if txn_type == "all" and txn_time == "all":
        year_list = {str(i):len(address_df.loc[str(i)]) for i in range(2015, 2023)}
        result = pd.DataFrame(list(year_list.items()), columns = ["year", "counts"])
        return result
    
    elif txn_type == "all" and txn_time != "all":
        year_list_output = {str(month):0 for month in range(1, 13)}
        month_count = {str(i):len(address_df.loc["{}-{}".format(txn_time, i)]) for i in range(1, 13)}
        year_list_output.update(month_count)
        result = pd.DataFrame(list(year_list_output.items()), columns = ["month", "counts"])
        return result
    
    elif txn_type != "all" and txn_time == "all":
        mask = address_df["txn_type"] == txn_type
        year_list = {str(i):len(address_df[mask].loc[str(i)]) for i in range(2015, 2023)}
        result = pd.DataFrame(list(year_list.items()), columns = ["year", "counts"])
        return result

    else:            
        year_list_output = {str(month):0 for month in range(1, 13)}

        for i in range(1, 13):
            tmp_df = address_df.loc["{}-{}".format(txn_time, i)]
            tmp_mask = tmp_df["txn_type"] == txn_type
            year_list_output[str(i)] = len(tmp_df[tmp_mask])

        result = pd.DataFrame(list(year_list_output.items()), columns = ["month", "counts"])
        return result

In [339]:
target_address = "0xd8dA6BF26964aF9D7eEd9e03E53415D37aA96045"
get_txns = Get_address_details(target_address)

In [340]:
pd.set_option("display.max_columns", None)
all_txn_result = get_txns.get_all_txns()

In [ ]:
def get_contract_df(address_df, txn_type, txn_year, txn_month):
    mask = all_txn_result["txn_type"] == txn_type
    tmp_df = all_txn_result[mask].loc[txn_year+"-"+txn_month]

    contarct_list = []
    contarct_set = set()
    if len(tmp_df) != 0:
        for data in tmp_df.iterrows(): 
            
            from_ads = data[1][7]
            to_ads = data[1][8]

            contarct_set.add(from_ads)
            contarct_set.add(to_ads)

            hash = data[1][3]
            value = int(data[1][9])/ETHER_VALUE
            if from_ads == target_address.lower():
                tmp_contract_dict = {"hash":hash, "ads":to_ads, "value":value}
            if to_ads == target_address.lower():
                tmp_contract_dict = {"hash":hash, "ads":from_ads, "value":value}
            contarct_list.append(tmp_contract_dict)

        contarct_set.remove(target_address.lower())
        contract_df = pd.DataFrame(contarct_list)
        contract_name = {}
        for i in contarct_set:
            name = get_contract_name(i)
            if name == "":
                contract_name[i] = "unknown"
            else:
                contract_name[i] = name
        contract_df.insert(0, "ContractName", "unknown")
        for ads in contract_name:
            # df.loc[df.grades>50,'result']='success'
            contract_df.loc[contract_df.ads == ads, "ContractName"] = contract_name[ads]
    return contract_df

In [367]:
txn_type = "internal"
txn_year = "2022"
txn_month = "05"

mask = all_txn_result["txn_type"] == txn_type

In [368]:
tmp_df = all_txn_result[mask].loc[txn_year + "-" + txn_month]
# tmp_df = all_txn_result[mask]

# Index(['blockNumber', 'txn_type', 'timeStamp', 'hash', 'nonce', 'blockHash',
#        'transactionIndex', 'from', 'to', 'value', 'gas', 'gasPrice', 'isError',
#        'txreceipt_status', 'input', 'contractAddress', 'cumulativeGasUsed',
#        'gasUsed', 'confirmations', 'methodId', 'functionName', 'type',
#        'traceId', 'errCode'],
#       dtype='object')

In [343]:
contarct_list = []
contarct_set = set()
if len(tmp_df) != 0:
    for data in tmp_df.iterrows(): 
        
        from_ads = data[1][7]
        to_ads = data[1][8]

        contarct_set.add(from_ads)
        contarct_set.add(to_ads)

        hash = data[1][3]
        value = int(data[1][9])/ETHER_VALUE
        if from_ads == target_address.lower():
            tmp_contract_dict = {"hash":hash, "ads":to_ads, "value":value}
        if to_ads == target_address.lower():
            tmp_contract_dict = {"hash":hash, "ads":from_ads, "value":value}
        contarct_list.append(tmp_contract_dict)

    contarct_set.remove(target_address.lower())
# print(contarct_set)

In [344]:
contract_df = pd.DataFrame(contarct_list)
# output
# 	                                             hash                                          ads	   value
# 0	0x1c318f84339d3a2354e797261cdfdf592d35477c5c28...	0x7e2d0fe0ffdd78c264f8d40d19acb7d04390c6e8	0.000200
# 1	0x55c749e2dfefb46060a455ea2037319c802c8f76008d...	0x7e2d0fe0ffdd78c264f8d40d19acb7d04390c6e8	0.000060

In [338]:
contract_name = {}
for i in contarct_set:
    name = get_contract_name(i)
    if name == "":
        contract_name[i] = "unknown"
    else:
        contract_name[i] = name

# output
# {'0xc9cd61c6f278dc6e4cd0f8fe6a8ad1d595b6ec8b': '',
#  '0xb671c3883307cf05bb3dff77a9754e87b4347195': 'HFConditionalTransfer',
#  '0x2136bbba2edca21afdddee838fff19ea70d10f03': 'EtherDelta',
#  '0xfc209f3506ee55ea9c14ebe6df5bd45c1338f085': 'Deed',...}

In [345]:
contract_df.insert(0, "ContractName", "unknown")

In [346]:
for ads in contract_name:
    # df.loc[df.grades>50,'result']='success'
    contract_df.loc[contract_df.ads == ads, "ContractName"] = contract_name[ads]
        

In [349]:
contract_df

,ContractName,hash,ads,value
0,unknown,0x1c318f84339d3a2354e797261cdfdf592d35477c5c28...,0x7e2d0fe0ffdd78c264f8d40d19acb7d04390c6e8,0.000200
1,unknown,0x55c749e2dfefb46060a455ea2037319c802c8f76008d...,0x7e2d0fe0ffdd78c264f8d40d19acb7d04390c6e8,0.000060
2,unknown,0x5f3564bdad98c20eb2c1aeff20d9b4530dc65bf9c627...,0x31e684ad5c33b741b4043bb28ece3b0c1bf9e4d4,0.000090
3,unknown,0x9795a2df6d1c486e9e3c02e65d60e1fc049345ee5a07...,0x31e684ad5c33b741b4043bb28ece3b0c1bf9e4d4,0.001000
4,unknown,0xc5fc74c372c3ba4317a982f06aeb6637b879ad069aea...,0x31e684ad5c33b741b4043bb28ece3b0c1bf9e4d4,0.000400
...,...,...,...,...
65,Proxy,0x68a5f55a8b81e644e1564575bacec34c86d08d555407...,0x220866b1a2219f40e72f5c628b65d54268ca3a9d,5000.000000
66,SwapRouter02,0x9846ec9e063cea51cc50bca3a8d52874bcd7029d072d...,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,0.473917
67,ETHRegistrarController,0x0c1e0b69e4af1815834db4fdc946e2a24f7a7625c992...,0x283af0b28c62c092c9727f1ee09c02ca627eb7f5,0.001395
68,SwapRouter02,0x6e1a6db2dfb8f96cbcc5520f3c06aaa9aeaa1ba4457a...,0x68b3465833fb72a70ecdf485e0e4c7bd8665fc45,611.831148


In [288]:
contract_list = []
for hash_ads in list(contarct_set):
    hash = hash_ads[0]
    ads = hash_ads[1]
    name = get_contract_name(ads)
    result = {name : ads}
    # print(name)
    if name == "":
        result = {"unknown" : ads}
        print(result)
    else:
        print(result)
    contract_list.append(result)

TypeError: string indices must be integers

In [284]:
for contra in contract_list:
    print(contra.values())
    print(tmp_df[tmp_df["from"] == contra.values()])
    break

dict_values(['0xc9cd61c6f278dc6e4cd0f8fe6a8ad1d595b6ec8b'])
Empty DataFrame
Columns: [blockNumber, txn_type, timeStamp, hash, nonce, blockHash, transactionIndex, from, to, value, gas, gasPrice, isError, txreceipt_status, input, contractAddress, cumulativeGasUsed, gasUsed, confirmations, methodId, functionName, type, traceId, errCode]
Index: []


In [234]:
get_contract_name("0x283af0b28c62c092c9727f1ee09c02ca627eb7f5")

''

In [173]:
for txn in get_txns.get_internal_txns():
    print("From :", txn["from"])
    print("To :", txn["to"])
    print("Value :", int(txn["value"])/ETHER_VALUE)
    print("Type :", txn["type"])
    print("Hash :", txn["hash"])
    print("="*10)

From : 0x7e2d0fe0ffdd78c264f8d40d19acb7d04390c6e8
To : 0xd8da6bf26964af9d7eed9e03e53415d37aa96045
Value : 0.0002
Type : call
Hash : 0x1c318f84339d3a2354e797261cdfdf592d35477c5c28e5f62d46dbc97aa84e38
From : 0x7e2d0fe0ffdd78c264f8d40d19acb7d04390c6e8
To : 0xd8da6bf26964af9d7eed9e03e53415d37aa96045
Value : 6e-05
Type : call
Hash : 0x55c749e2dfefb46060a455ea2037319c802c8f76008dc8a4d3fdf2c99dc6201c
From : 0x31e684ad5c33b741b4043bb28ece3b0c1bf9e4d4
To : 0xd8da6bf26964af9d7eed9e03e53415d37aa96045
Value : 9e-05
Type : call
Hash : 0x5f3564bdad98c20eb2c1aeff20d9b4530dc65bf9c6274d4476795a13d100f57c
From : 0x31e684ad5c33b741b4043bb28ece3b0c1bf9e4d4
To : 0xd8da6bf26964af9d7eed9e03e53415d37aa96045
Value : 0.001
Type : call
Hash : 0x9795a2df6d1c486e9e3c02e65d60e1fc049345ee5a0703534c9bbcaa7fdc4d85
From : 0x31e684ad5c33b741b4043bb28ece3b0c1bf9e4d4
To : 0xd8da6bf26964af9d7eed9e03e53415d37aa96045
Value : 0.0004
Type : call
Hash : 0xc5fc74c372c3ba4317a982f06aeb6637b879ad069aea0c8825721e34dddb67c8
From : 0

In [ ]:
get_txns.get_all_txns()

In [146]:
get_txns = Get_address_details(address)
result = get_txns.get_all_txns_by_datetime()
result

AttributeError: 'Get_address_details' object has no attribute 'get_all_txns_by_datetime'

In [ ]:
def txns_by_year(address_result_dict):
    datetime_list = list(address_result_dict.keys())
    yeartime_dict = {}
    
    for year_day in address_result_dict:
        year = year_day.split("-")[0]
        if year in yeartime_dict:
            yeartime_dict[year] += 1
        else:
            yeartime_dict[year] = 1
    
    return yeartime_dict
        

In [ ]:
txns_by_year(result)

In [ ]:
def txn_in_specific_year(target_year, address_result_dict):
    
    month_time_dict = {}
    
    for year_day in address_result_dict:
        year, month = year_day.split("-")[0], year_day.split("-")[1]
        if year == target_year:
            if month in month_time_dict:
                month_time_dict[month] += 1
            else:
                month_time_dict[month] = 1
        else:
            pass
        
    return month_time_dict

In [ ]:
txn_in_specific_year("2019", result)

In [ ]:
df = pd.DataFrame(list(result.items()),
                   columns=['datetime', 'counts'])

In [ ]:
df

In [ ]:
list(df.items())[0][1].tolist()

In [ ]:
list(df.items())[1][1].tolist()

In [ ]:
len(df)

In [ ]:
# example for get account balance url
"""
https://api.etherscan.io/api
   ?module=account
   &action=balance
   &address=0xde0b295669a9fd93d5f28d9ec85e40f4cb697bae
   &tag=latest
   &apikey=YourApiKeyToken
"""

### Create api url

In [ ]:
def make_api_url(module, action, address, **kwargs):
    url = BASE_URL + f"?module={module}&action={action}&address={address}&apikey={API_KEY}"

    for key, value in kwargs.items():
        url += f"&{key}={value}"

    return url

### Get account balance by address

In [ ]:
def get_account_balance(address):
    balance_url = make_api_url("account", "balance", address, tag="latest")
    response = get(balance_url)
    data = response.json()

    value = int(data["result"]) / ETHER_VALUE
    return value

In [ ]:
account_balance = get_account_balance(address)
print(account_balance)

In [ ]:
# Example for get all txns record from address
"""
https://api.etherscan.io/api
   ?module=account
   &action=txlist
   &address=0xc5102fE9359FD9a28f877a67E36B0F050d81a3CC
   &startblock=0
   &endblock=99999999
   &page=1
   &offset=10
   &sort=asc
   &apikey=YourApiKeyToken
"""

In [ ]:
def get_txns(address):
    """
    Get all txns 
    """
    transactions_url = make_api_url("account", "txlist", address, startblock=0, endblock=99999999, page=1, offset=10000, sort="asc")
    response = get(transactions_url)
    data = response.json()["result"]

    internal_tx_url = make_api_url("account", "txlistinternal", address, startblock=0, endblock=99999999, page=1, offset=10000, sort="asc")
    response2 = get(internal_tx_url)
    data2 = response2.json()["result"]

    data.extend(data2)
    data.sort(key=lambda x: int(x['timeStamp']))
    
    return data

In [ ]:
def get_txns_sum_by_datetime(address):
    address_data = get_txns(address)
    txns_by_datetime = {}
    
    for txn_data in address_data:
        time = str(datetime.fromtimestamp(int(txn_data["timeStamp"]))).split(" ")[0]
        
        if time in txns_by_datetime:
            txns_by_datetime[time] += 1
        else:
            txns_by_datetime[time] = 1
    return txns_by_datetime

In [ ]:
def get_normal_txns_by_datetime(address):
    
    """
    Get normal txns by date time
    """
    
    txns_by_datetime = {}
    
    transactions_url = make_api_url("account", "txlist", address, startblock=0, endblock=99999999, page=1, offset=10000, sort="asc")
    response = get(transactions_url)
    data = response.json()["result"]
    
    data.sort(key=lambda x: int(x['timeStamp']))
    
    for d in data:
        time = str(datetime.fromtimestamp(int(d["timeStamp"]))).split(" ")[0]
        
        if time in txns_by_datetime:
            txns_by_datetime[time] += 1
        else:
            txns_by_datetime[time] = 1
        
    return txns_by_datetime
    

In [ ]:
noraml_txns = get_normal_txns_by_datetime(address)

In [ ]:
noraml_txns

In [ ]:
def get_internal_txns_by_datetime(address):
    
    """
    Get normal txns by date time
    """
    
    txns_by_datetime = {}
    
    transactions_url = make_api_url("account", "txlistinternal", address, startblock=0, endblock=99999999, page=1, offset=10000, sort="asc")
    response = get(transactions_url)
    data = response.json()["result"]
    
    data.sort(key=lambda x: int(x['timeStamp']))
    
    for d in data:
        time = str(datetime.fromtimestamp(int(d["timeStamp"]))).split(" ")[0]
        
        if time in txns_by_datetime:
            txns_by_datetime[time] += 1
        else:
            txns_by_datetime[time] = 1
        
    return txns_by_datetime

In [ ]:
internal_txns_datetime = get_internal_txns_by_datetime(address2)

In [ ]:
internal_txns_datetime

In [ ]:
tick_spacing = len(internal_txns_datetime)/7 # x軸密集度

fig = plt.figure(figsize=(10, 8)) # 寬10 長8
gs = GridSpec(1, 1) # 設立1x1的網格

ax = plt.subplot(gs[0, 0]) # 子圖ax在網格的[0, 0]
ax.xaxis.set_major_locator(mticker.MultipleLocator(tick_spacing))

ax.plot_date(list(internal_txns_datetime.keys()), list(internal_txns_datetime.values()), linestyle ='solid')

### Ploting txns (nomal) sum by datetime

In [ ]:
normal_txns_datetime = get_normal_txns_by_datetime(address2)

In [ ]:
tick_spacing = len(normal_txns_datetime)/7 # x軸密集度

fig = plt.figure(figsize=(10, 8)) # 寬10 長8
gs = GridSpec(1, 1) # 設立1x1的網格

ax = plt.subplot(gs[0, 0]) # 子圖ax在網格的[0, 0]
ax.xaxis.set_major_locator(mticker.MultipleLocator(tick_spacing))

ax.plot_date(list(normal_txns_datetime.keys()), list(normal_txns_datetime.values()), linestyle ='solid')

### Ploting txns (nomal and internal) sum by datetime

In [ ]:
txns_datetime = get_txns_sum_by_datetime(address2)

In [ ]:
tick_spacing = len(txns_datetime)/7 # x軸密集度

fig = plt.figure(figsize=(10, 8)) # 寬10 長8
gs = GridSpec(1, 1) # 設立1x1的網格

ax = plt.subplot(gs[0, 0]) # 子圖ax在網格的[0, 0]
ax.xaxis.set_major_locator(mticker.MultipleLocator(tick_spacing))

ax.plot_date(list(txns_datetime.keys()), list(txns_datetime.values()), linestyle ='solid')